Classification of Cars using Feed Forward Network(Perceptron)

Data Preprocessing Part

In [1]:
import os
import glob
import pickle
import random
import numpy as np
import cv2
from datetime import datetime

In [2]:
directories = [name for name in glob.glob('Dataset/*') if os.path.isdir(name)]
directories

['Dataset\\AM General Hummer SUV 2000',
 'Dataset\\Aston Martin V8 Vantage Convertible 2012',
 'Dataset\\Aston Martin V8 Vantage Coupe 2012',
 'Dataset\\Aston Martin Virage Convertible 2012',
 'Dataset\\Aston Martin Virage Coupe 2012',
 'Dataset\\Bentley Arnage Sedan 2009',
 'Dataset\\Bentley Continental Flying Spur Sedan 2007',
 'Dataset\\Bentley Continental GT Coupe 2007',
 'Dataset\\Bentley Continental GT Coupe 2012',
 'Dataset\\Bentley Continental Supersports Conv. Convertible 2012',
 'Dataset\\Bentley Mulsanne Sedan 2011',
 'Dataset\\Bugatti Veyron 16.4 Convertible 2009',
 'Dataset\\Bugatti Veyron 16.4 Coupe 2009',
 'Dataset\\Ferrari 458 Italia Convertible 2012',
 'Dataset\\Ferrari 458 Italia Coupe 2012',
 'Dataset\\Ferrari California Convertible 2012',
 'Dataset\\Ferrari FF Coupe 2012',
 'Dataset\\Rolls-Royce Ghost Sedan 2012',
 'Dataset\\Rolls-Royce Phantom Drophead Coupe Convertible 2012',
 'Dataset\\Rolls-Royce Phantom Sedan 2012']

In [3]:
len(directories)

20

In [4]:
classes = []
for directory in directories:
    classes.append(directory.split('\\')[1])
classes

['AM General Hummer SUV 2000',
 'Aston Martin V8 Vantage Convertible 2012',
 'Aston Martin V8 Vantage Coupe 2012',
 'Aston Martin Virage Convertible 2012',
 'Aston Martin Virage Coupe 2012',
 'Bentley Arnage Sedan 2009',
 'Bentley Continental Flying Spur Sedan 2007',
 'Bentley Continental GT Coupe 2007',
 'Bentley Continental GT Coupe 2012',
 'Bentley Continental Supersports Conv. Convertible 2012',
 'Bentley Mulsanne Sedan 2011',
 'Bugatti Veyron 16.4 Convertible 2009',
 'Bugatti Veyron 16.4 Coupe 2009',
 'Ferrari 458 Italia Convertible 2012',
 'Ferrari 458 Italia Coupe 2012',
 'Ferrari California Convertible 2012',
 'Ferrari FF Coupe 2012',
 'Rolls-Royce Ghost Sedan 2012',
 'Rolls-Royce Phantom Drophead Coupe Convertible 2012',
 'Rolls-Royce Phantom Sedan 2012']

In [5]:
len(classes)

20

In [6]:
feature_data = []
for directory in directories:
    paths = glob.glob(directory+'/*.jpg')
    current_class = directory.split('\\')[1]
    for path in paths:
        feature_dictionary = {}
        feature_dictionary['path'] = path
        feature_data.append((current_class,feature_dictionary))

len(feature_data)

1575

In [7]:
feature_data

[('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00076.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00163.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00457.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00462.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00522.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00684.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00707.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00773.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00887.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Dataset\\AM General Hummer SUV 2000\\00946.jpg'}),
 ('AM General Hummer SUV 2000',
  {'path': 'Datase

In [8]:
with open('feature_data.pkl','wb') as f:
    pickle.dump(feature_data,f)
with open('classes.pkl','wb') as f:
    pickle.dump(classes,f)
    
with open('feature_data.pkl','rb') as f:
    feature_data = pickle.load(f)
with open('classes.pkl','rb') as f:
    classes = pickle.load(f)

Designing the Network

Training Algorithm Begins

In [9]:
bias = 1
Train_Test_Ratio = 0.9
Iterations = 30
lr = 0.05

In [10]:
def sigmoid(x):
    return (1/1+np.exp(-x))

def ReLu(x):
    return max(0,x)              

In [11]:
class MultiClassPerceptron:
    
    accuracy = 0
    def __init__(self,classes,feature_data,train_test_ratio=Train_Test_Ratio,iterations=Iterations):
        self.classes = classes
        self.feature_data = feature_data
        self.ratio = train_test_ratio
        self.iterations = iterations
        
        random.shuffle(self.feature_data)
        self.train_set = self.feature_data[:int(len(self.feature_data)*self.ratio)]
        self.test_set = self.feature_data[int(len(self.feature_data)*self.ratio):]
        
        self.weight_vectors = {c:np.array([0.0 for _ in range(9601)]) for c in self.classes}
        
    def train(self):

                
        #####
        
        for _ in range(self.iterations):
            i = 0
            for category,feature_dictionary in self.train_set:
                i+=1
                fract = i / len(self.train_set)
                print('{:>2.2%}'.format(fract), end="\r")

                image = cv2.imread(feature_dictionary['path'])
                dim = (120,80)
                resized_image = cv2.resize(image,dim,cv2.INTER_AREA)

                input_array = []
                input_array = [np.mean(resized_image[i][j]/255) for i in range(resized_image.shape[0])
                              for j in range(resized_image.shape[1])]
                input_array.append(bias)
                input_vector = np.array(input_array)
                
                arg_max,predicted_class = 0,self.classes[0]
                
                for index in self.classes:
                    current_activation = np.dot(input_vector,self.weight_vectors[index])
                    current_activation = sigmoid(current_activation)
                    if current_activation >= arg_max:
                        arg_max,predicted_class = current_activation,index
                        
                if not(category == predicted_class):
                    self.weight_vectors[category] += [weight*lr for weight in input_vector]
                    self.weight_vectors[predicted_class] -= [weight*lr for weight in input_vector]
                    
            print(str(datetime.now().strftime("%H:%M:%S")) + '\tCompleted epoch:    {}'.format(_+1), end="")
            self.calculate_accuracy()
                    
         
    def predict(self,feature_dictionary):
        
        img = cv2.imread(feature_dictionary['path'])
        dim = (120,80)
        resized_image = cv2.resize(img,dim,cv2.INTER_AREA)
        
        input_array = []
        input_array = [np.mean(resized_image[i][j]/255) for i in range(resized_image.shape[0])
                      for j in range(resized_image.shape[1])]
        
        
        input_array.append(bias)
        feature_vector = np.array(input_array)
        
        arg_max,predicted_class = 0,self.classes[0]
                
        for index in self.classes:
            current_activation = np.dot(feature_vector,self.weight_vectors[index])
            current_activation = sigmoid(current_activation)
            if current_activation >= arg_max:
                arg_max,predicted_class = current_activation,index
                
        return predicted_class
    
    def test_random_data(self):
        item = random.choice(self.test_set)
        print("Actual Class :" + item[0])
        pred_class = self.predict(item[1])
        print("\tPredicted Class :" + pred_class)
        
        
    def calculate_accuracy(self):
        
        correct,incorrect = 0,0
        random.shuffle(self.feature_data)
        self.test_set = self.feature_data[int(len(self.feature_data)*self.ratio):]
        for feature_dictionary in self.test_set:
            actual_class = feature_dictionary[0]
            predicted_class = self.predict(feature_dictionary[1])
            
            if actual_class == predicted_class:
                correct += 1
            else:
                incorrect += 1
                
#         print("ACCURACY")
        print("\tAccuracy of FF Model is ",format(((correct * 1.0) / ((correct + incorrect) * 1.0)*100), '.2f'))
        
    
    def select_random_picture(self):
        
        s = glob.glob(os.getcwd() + '\\Testing_Sample/*')
        
        img = cv2.imread(s[0])
        dim = (120,80)
        resized_image = cv2.resize(img,dim,cv2.INTER_AREA)
        
        input_array = []
        input_array = [np.mean(resized_image[i][j]/255) for i in range(resized_image.shape[0])
                      for j in range(resized_image.shape[1])]
        
        
        input_array.append(bias)
        feature_vector = np.array(input_array)
        
        
        arg_max,predicted_class = 0,self.classes[0]
    
        for index in self.classes:
            current_activation = np.dot(feature_vector,self.weight_vectors[index])
            if current_activation >= arg_max:
                arg_max,predicted_class = current_activation,index
                
        print("The Predicted Class for the Sample Image is")
        print(predicted_class)

In [12]:
if __name__ == "__main__":
    classifier = MultiClassPerceptron(classes,feature_data)
    classifier.train()
#     classifier.calculate_accuracy()
#     classifier.test_random_data()

C:\Users\Administer\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


15:32:27	Completed epoch:    1	Accuracy of FF Model is  5.06
15:35:10	Completed epoch:    2	Accuracy of FF Model is  1.27
15:37:52	Completed epoch:    3	Accuracy of FF Model is  5.70
15:40:55	Completed epoch:    4	Accuracy of FF Model is  5.70
15:43:44	Completed epoch:    5	Accuracy of FF Model is  6.96
15:46:25	Completed epoch:    6	Accuracy of FF Model is  10.13
15:49:05	Completed epoch:    7	Accuracy of FF Model is  6.33
15:51:45	Completed epoch:    8	Accuracy of FF Model is  5.06
15:54:30	Completed epoch:    9	Accuracy of FF Model is  4.43
15:57:20	Completed epoch:    10	Accuracy of FF Model is  5.70
16:00:13	Completed epoch:    11	Accuracy of FF Model is  12.03
16:02:55	Completed epoch:    12	Accuracy of FF Model is  3.80
16:05:28	Completed epoch:    13	Accuracy of FF Model is  4.43
16:08:06	Completed epoch:    14	Accuracy of FF Model is  5.70
16:10:36	Completed epoch:    15	Accuracy of FF Model is  5.06
16:13:06	Completed epoch:    16	Accuracy of FF Model is  4.43
16:15:36	Comple

In [13]:
for _ in range(15):
    classifier.test_random_data()
    print('\n')

Actual Class :Rolls-Royce Phantom Drophead Coupe Convertible 2012
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :AM General Hummer SUV 2000
	Predicted Class :Rolls-Royce Phantom Sedan 2012




C:\Users\Administer\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Actual Class :Aston Martin Virage Convertible 2012
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Bugatti Veyron 16.4 Coupe 2009
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :AM General Hummer SUV 2000
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Bentley Arnage Sedan 2009
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Aston Martin Virage Coupe 2012
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Bentley Continental GT Coupe 2012
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Ferrari 458 Italia Coupe 2012
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Bugatti Veyron 16.4 Coupe 2009
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Rolls-Royce Phantom Sedan 2012
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Ferrari 458 Italia Coupe 2012
	Predicted Class :Rolls-Royce Phantom Sedan 2012


Actual Class :Rolls-Royce Phantom Sedan 20

In [15]:
classifier.select_random_picture()

The Predicted Class for the Sample Image is
Aston Martin V8 Vantage Convertible 2012
